# 1)-Importing key Modules



In [0]:
#support both Python 2 and Python 3 with minimal overhead.
from __future__ import absolute_import, division, print_function

# I am an engineer. I care only about error not warning. So, let's be maverick and ignore warnings.
import warnings
warnings.filterwarnings('ignore')

In [2]:
! pip install dynet

In [0]:
from collections import defaultdict
import time
import random
import dynet as dy
import numpy as np

In [4]:
! pip install version_information

  Created wheel for version-information: filename=version_information-1.0.3-cp36-none-any.whl size=3880 sha256=f176b54cd894b7bbad5a4d85bda1110b2897500a969ae6f27d77c88e8b333430
  Stored in directory: /root/.cache/pip/wheels/1f/4c/b3/1976ac11dbd802723b564de1acaa453a72c36c95827e576321
Successfully built version-information


In [5]:
# first install: pip install version_information
%reload_ext version_information
%version_information pandas,dynet,numpy

Software versions
Python 3.6.8 64bit [GCC 8.3.0]
IPython 5.5.0
OS Linux 4.14.137+ x86_64 with Ubuntu 18.04 bionic
pandas 0.25.3
dynet 2.0.3
numpy 1.17.3
Thu Nov 07 21:32:17 2019 UTC

# 2)- Loading Data

In [0]:
# Functions to read in the corpus
w2i = defaultdict(lambda: len(w2i))
t2i = defaultdict(lambda: len(t2i))
UNK = w2i["<unk>"]
def read_dataset(filename):
  with open(filename, "r") as f:
    for line in f:
      tag, words = line.lower().strip().split(" ||| ")
      yield ([w2i[x] for x in words.split(" ")], t2i[tag])

In [0]:
# Read in the data
train = list(read_dataset("train.txt"))
w2i = defaultdict(lambda: UNK, w2i)
dev = list(read_dataset("test.txt"))

In [12]:
print(len(w2i))
print(len(dev))

18649
2210


In [0]:
nwords = len(w2i)
ntags = len(t2i)

In [15]:
train[0][1]

0

In [0]:
# Start DyNet and define trainer
model = dy.Model()
trainer = dy.AdamTrainer(model)

In [0]:
# Define the model
W_sm = model.add_lookup_parameters((nwords, ntags)) # Word weights
b_sm = model.add_parameters((ntags))                # Softmax bias

In [0]:
# A function to calculate scores for one value
def calc_scores(words):
  dy.renew_cg()
  score = dy.esum([dy.lookup(W_sm, x) for x in words])
  b_sm_exp = dy.parameter(b_sm)
  return score + b_sm_exp

# 3)- Train model

In [19]:
for ITER in range(100):
  # Perform training
  random.shuffle(train)
  train_loss = 0.0
  start = time.time()
  for words, tag in train:
    my_loss = dy.pickneglogsoftmax(calc_scores(words), tag)
    train_loss += my_loss.value()
    my_loss.backward()
    trainer.update()
  print("iter %r: train loss/sent=%.4f, time=%.2fs" % (ITER, train_loss/len(train), time.time()-start))
  # Perform testing
  test_correct = 0.0
  for words, tag in dev:
    scores = calc_scores(words).npvalue()
    predict = np.argmax(scores)
    if predict == tag:
      test_correct += 1
  print("iter %r: test acc=%.4f" % (ITER, test_correct/len(dev)))

The dy.parameter(...) call is now DEPRECATED.
        There is no longer need to explicitly add parameters to the computation graph.
        Any used parameter will be added automatically.
iter 0: train loss/sent=1.5173, time=0.23s
iter 0: test acc=0.3597
iter 1: train loss/sent=1.3385, time=0.22s
iter 1: test acc=0.3765
iter 2: train loss/sent=1.2355, time=0.22s
iter 2: test acc=0.3873
iter 3: train loss/sent=1.1567, time=0.23s
iter 3: test acc=0.3905
iter 4: train loss/sent=1.0920, time=0.23s
iter 4: test acc=0.4036
iter 5: train loss/sent=1.0381, time=0.22s
iter 5: test acc=0.4045
iter 6: train loss/sent=0.9913, time=0.22s
iter 6: test acc=0.4104
iter 7: train loss/sent=0.9498, time=0.21s
iter 7: test acc=0.4063
iter 8: train loss/sent=0.9129, time=0.23s
iter 8: test acc=0.4113
iter 9: train loss/sent=0.8799, time=0.21s
iter 9: test acc=0.4090
iter 10: train loss/sent=0.8496, time=0.22s
iter 10: test acc=0.4136
iter 11: train loss/sent=0.8224, time=0.21s
iter 11: test acc=0.4181
ite